## Movies

Série de requêtes sur l'index "movies".
- Total number of reviewed movies
- Total number of reviews
- Total number of journalists
- Number of reviews by journalist
- Number of reviews per year
- Number of reviews by journalist and per year
- Number of MPAA categories
- Number of films per MPAA category

In [46]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Total number of reviewed movies

query_body = {
  "size": 0,
  "aggs": {
    "total_films": {
      "cardinality": {
        "field": "display_title.keyword"
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/3061637097.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 12,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'total_films': {'value': 8559}}}

In [47]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Total number of reviews

query_body = {
  "size": 0,
  "aggs": {
    "total_reviews": {
      "cardinality": {
        "field": "headline.keyword"
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/1967684515.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'total_reviews': {'value': 8720}}}

In [48]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Total number of journalists

query_body = {
  "size": 0,
  "aggs": {
    "total_journalists": {
      "cardinality": {
        "field": "byline.keyword"
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/2329979850.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 3,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'total_journalists': {'value': 104}}}

In [49]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Number of reviews by journalist

query_body = {
  "size": 0,
  "aggs": {
    "reviews_per_journalist": {
      "terms": {
        "field": "byline.keyword",
        "size": 10
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/4359584.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'reviews_per_journalist': {'doc_count_error_upper_bound': 86,
   'sum_other_doc_count': 3588,
   'buckets': [{'key': 'JEANNETTE CATSOULIS', 'doc_count': 855},
    {'key': 'MANOHLA DARGIS', 'doc_count': 717},
    {'key': 'STEPHEN HOLDEN', 'doc_count': 593},
    {'key': 'A. O. SCOTT', 'doc_count': 562},
    {'key': 'BEN KENIGSBERG', 'doc_count': 542},
    {'key': 'GLENN KENNY', 'doc_count': 495},
    {'key': 'NEIL GENZLINGER', 'doc_count': 416},
    {'key': 'NICOLAS RAPOLD', 'doc_count': 348},
    {'key': 'Glenn Kenny', 'doc_count': 345},
    {'key': 'ANDY WEBSTER', 'doc_count': 319}]}}}

In [50]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Number of MPAA categories

query_body = {
  "size": 0,
  "aggs": {
    "total_films": {
      "cardinality": {
        "field": "mpaa_rating"
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/4054417190.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'total_films': {'value': 19}}}

In [51]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Number of movies per MPAA category

query_body = {
  "size": 0,
  "aggs": {
    "films_by_mpaa_rating": {
      "terms": {
        "field": "mpaa_rating",
        "size": 10
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/4155455540.py:17: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 1,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'films_by_mpaa_rating': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 23,
   'buckets': [{'key': '', 'doc_count': 3994},
    {'key': 'R', 'doc_count': 2235},
    {'key': 'PG-13', 'doc_count': 1154},
    {'key': 'Not Rated', 'doc_count': 590},
    {'key': 'PG', 'doc_count': 468},
    {'key': 'Unrated', 'doc_count': 157},
    {'key': 'TV-MA', 'doc_count': 82},
    {'key': 'TV-14', 'doc_count': 32},
    {'key': 'G', 'doc_count': 30},
    {'key': 'TV-PG', 'doc_count': 15}]}}}

In [52]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Number of reviews per year

query_body = {
  "size": 0,
  "aggs": {
    "reviews_by_year": {
      "date_histogram": {
        "field": "publication_date",
        "calendar_interval": "year",
        "format": "yyyy",
        "order": {
          "_key": "desc"
        }
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/4160182980.py:21: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'reviews_by_year': {'buckets': [{'key_as_string': '2023',
     'key': 1672531200000,
     'doc_count': 370},
    {'key_as_string': '2022', 'key': 1640995200000, 'doc_count': 766},
    {'key_as_string': '2021', 'key': 1609459200000, 'doc_count': 769},
    {'key_as_string': '2020', 'key': 1577836800000, 'doc_count': 655},
    {'key_as_string': '2019', 'key': 1546300800000, 'doc_count': 696},
    {'key_as_string': '2018', 'key': 1514764800000, 'doc_count': 690},
    {'key_as_string': '2017', 'key': 1483228800000, 'doc_count': 741},
    {'key_as_string': '2016', 'key': 1451606400000, 'doc_count': 774},
    {'key_as_string': '2015', 'key': 1420070400000, 'doc_count': 776},
    {'key_as_string': '2014', 'key': 1388534400000, 'doc_count': 871},
    {'key_as_string': '2013', 'key': 1356

In [53]:
from elasticsearch import Elasticsearch
es = Elasticsearch(hosts = "http://@localhost:9200")

#Number of reviews by journalist and per year

query_body = {
  "size": 0,
  "aggs": {
    "reviews_by_journalist": {
      "terms": {
        "field": "byline.keyword",
        "size": 10
      },
      "aggs": {
        "reviews_by_year": {
          "date_histogram": {
            "field": "publication_date",
            "calendar_interval": "year",
            "format": "yyyy",
            "order": {
              "_key": "asc"
            }
          }
        }
      }
    }
  }
}
result = es.search(index="movies", body=query_body)
result.body

/var/folders/0l/46n5zz7920jg0lld8ddcxvv40000gn/T/ipykernel_65046/4115964872.py:29: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result = es.search(index="movies", body=query_body)


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 2, 'successful': 2, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 8780, 'relation': 'eq'},
  'max_score': None,
  'hits': []},
 'aggregations': {'reviews_by_journalist': {'doc_count_error_upper_bound': 86,
   'sum_other_doc_count': 3588,
   'buckets': [{'key': 'JEANNETTE CATSOULIS',
     'doc_count': 855,
     'reviews_by_year': {'buckets': [{'key_as_string': '2011',
        'key': 1293840000000,
        'doc_count': 26},
       {'key_as_string': '2012', 'key': 1325376000000, 'doc_count': 109},
       {'key_as_string': '2013', 'key': 1356998400000, 'doc_count': 129},
       {'key_as_string': '2014', 'key': 1388534400000, 'doc_count': 130},
       {'key_as_string': '2015', 'key': 1420070400000, 'doc_count': 80},
       {'key_as_string': '2016', 'key': 1451606400000, 'doc_count': 71},
       {'key_as_string': '2017', 'key': 1483228800000, 'doc_count': 109},
       {'key_as_string': '2018', 'key': 1514764800000, 'doc_count'